In [49]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import pandas as pd
import os

In [50]:
# Set environment variables for reproducibility
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms(True, warn_only=True)

os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

torch.manual_seed(40)
torch.cuda.manual_seed_all(40)

In [51]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
device

'cuda'

In [52]:
def train_model(model, train_loader, test_loader, num_epochs, loss_function, optimizer, batch_size, classes):
    for epoch in range(num_epochs):
        for i, (image, label) in enumerate(train_loader):
            image = image.to(device)
            label = label.to(device)
        
            optimizer.zero_grad()
            output = model(image)
            loss = loss_function(output, label)
            loss.backward()
            optimizer.step()
        
            if (i+1)%2500==0:
                print(f'Epoch {epoch+1}, Step {i+1}, Loss= {loss.item():.4f}')
    print('Training Complete')

    with torch.no_grad():
        n_correct = 0
        n_samples = 0
        n_class_correct = [0 for _ in range(10)]
        n_class_samples = [0 for _ in range(10)]
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            output = model(images)
            _, predicted = torch.max(output, 1)
            n_samples += labels.size(0)
            n_correct += (predicted==labels).sum().item()
        
            for i in range(batch_size):
                label = labels[i]
                pred = predicted[i]
                if (label==pred):
                    n_class_correct[label] += 1
                n_class_samples[label] += 1
    
    print(f'Accuracy of the network: {100 * n_correct/n_samples} %')

    for i in range(10):
        acc = 100.0*n_class_correct[i]/n_class_samples[i]
        print(f'Accuracy of {classes[i]} : {acc:.3f}%')

In [53]:
epochs = 4
batch_size = 4
lr = 1e-3

In [54]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [55]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16,32,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(32*8*8, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )
    def forward(self, x):
        return self.model(x)

In [56]:
model = CNN().to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr)

In [57]:
train_model(model, train_loader, test_loader, epochs, loss_function, optimizer, batch_size, classes)

Epoch 1, Step 2500, Loss= 2.3134
Epoch 1, Step 5000, Loss= 1.9319
Epoch 1, Step 7500, Loss= 0.8081
Epoch 1, Step 10000, Loss= 0.9845
Epoch 1, Step 12500, Loss= 1.4370
Epoch 2, Step 2500, Loss= 1.1437
Epoch 2, Step 5000, Loss= 1.0243
Epoch 2, Step 7500, Loss= 1.2530
Epoch 2, Step 10000, Loss= 1.1213
Epoch 2, Step 12500, Loss= 1.1019
Epoch 3, Step 2500, Loss= 1.3773
Epoch 3, Step 5000, Loss= 1.8691
Epoch 3, Step 7500, Loss= 0.2945
Epoch 3, Step 10000, Loss= 0.8074
Epoch 3, Step 12500, Loss= 0.8841
Epoch 4, Step 2500, Loss= 1.3149
Epoch 4, Step 5000, Loss= 0.7557
Epoch 4, Step 7500, Loss= 0.9237
Epoch 4, Step 10000, Loss= 0.3721
Epoch 4, Step 12500, Loss= 0.9601
Training Complete
Accuracy of the network: 61.94 %
Accuracy of plane : 65.600%
Accuracy of car : 74.500%
Accuracy of bird : 46.100%
Accuracy of cat : 51.300%
Accuracy of deer : 51.400%
Accuracy of dog : 45.200%
Accuracy of frog : 69.200%
Accuracy of horse : 73.800%
Accuracy of ship : 75.000%
Accuracy of truck : 67.300%
